In [19]:
import sqlite3

conn = sqlite3.connect('worldometer.db')

cur = conn.cursor()

In [20]:

cur.execute('DROP TABLE IF EXISTS country;')

1. City

In [12]:
import pandas as pd

file_path = 'city-to-country.csv'
df = pd.read_csv(file_path)

df_clean = df.dropna(subset=['city_ascii'])

df_unique = df_clean.drop_duplicates(subset=['city_ascii'], keep='first')

output_path = 'city-to-country-unique.csv'
df_unique.to_csv(output_path, index=False)

df_unique.head(), df_unique.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44155 entries, 0 to 47867
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   city        44155 non-null  object 
 1   city_ascii  44155 non-null  object 
 2   lat         44155 non-null  float64
 3   lng         44155 non-null  float64
 4   country     44155 non-null  object 
 5   iso2        44122 non-null  object 
 6   iso3        44155 non-null  object 
 7   admin_name  43967 non-null  object 
 8   capital     12297 non-null  object 
 9   population  43963 non-null  float64
 10  id          44155 non-null  int64  
dtypes: float64(3), int64(1), object(7)
memory usage: 4.0+ MB


(        city city_ascii      lat       lng    country iso2 iso3   admin_name  \
 0      Tokyo      Tokyo  35.6897  139.6922      Japan   JP  JPN        Tōkyō   
 1    Jakarta    Jakarta  -6.1750  106.8275  Indonesia   ID  IDN      Jakarta   
 2      Delhi      Delhi  28.6100   77.2300      India   IN  IND        Delhi   
 3  Guangzhou  Guangzhou  23.1300  113.2600      China   CN  CHN    Guangdong   
 4     Mumbai     Mumbai  19.0761   72.8775      India   IN  IND  Mahārāshtra   
 
    capital  population          id  
 0  primary  37732000.0  1392685764  
 1  primary  33756000.0  1360771077  
 2    admin  32226000.0  1356872604  
 3    admin  26940000.0  1156237133  
 4    admin  24973000.0  1356226629  ,
 None)

In [13]:
cur.execute('CREATE TABLE city( city VARCHAR(255) NOT NULL PRIMARY KEY, country VARCHAR(255) REFERENCES country(country) )')

In [14]:
city_path = 'city.csv'
df_city = df_unique[['city_ascii', 'country']]
df_city.to_csv(city_path, index=False)

2. Country

In [21]:
cur.execute('CREATE TABLE country(country VARCHAR(255) NOT NULL PRIMARY KEY)')

In [16]:
country_path = 'country.csv'

df_country = df_unique[['country']]
df_country = df_country.drop_duplicates()
df_country.to_csv(country_path, index=False)

3. Freedom

In [28]:
# þanks to Mark Kortink for this snippet (https://stackoverflow.com/questions/57052857/how-to-import-excel-data-into-sqlite-database-in-python)
fiw = pd.read_excel('fiw-data.xlsx',sheet_name = 'FIW13-23') 

freedom = fiw[fiw['Edition'].astype(str).str.contains('2023')]
freedom = freedom[['Country/Territory', 'Total']]
freedom.to_csv('freedom_in_world.csv', index=False)

In [12]:
cur.execute('CREATE TABLE freedom_in_world(country VARCHAR(255) NOT NULL PRIMARY KEY REFERENCES country(country),freedom_score INT NOT NULL  )')

4. Companies

In [35]:
companies = pd.read_json('companies.json')
companies = companies.dropna(subset=['bibtex_key', 'company_name', 'hq_city'])

def clean_hq_city(hq_city):
    if isinstance(hq_city, list):
        return hq_city[0]
    hq_city = hq_city.replace("['", "").replace("']", "").replace("'", "").replace("[", "").replace("]", "")
    return hq_city.split(',')[0]

companies['hq_city'] = companies['hq_city'].apply(clean_hq_city)
companies = companies[['bibtex_key', 'company_name', 'hq_city']]

companies_path = 'companies.csv'
companies.to_csv(companies_path, index=False)

In [14]:
cur.execute('CREATE TABLE companies (bibtex_key VARCHAR(43) NOT NULL PRIMARY KEY,company_name VARCHAR(43) NOT NULL,hq_city VARCHAR(31) NOT NULL REFERENCES city(city));')

Populating is done in seperate SQL files

In [15]:
conn.commit()
conn.close()